In [2]:
# Capital Spending Project: Joining on CPDB
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re
import datetime
import matplotlib.font_manager
import time

# Capital Spending Project: Joining on CPDB

In [3]:
# --- checkbook nyc data
file_names = ['../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_0.csv', 
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_1.csv',
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_2.csv',
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_3.csv']
df0 = pd.read_csv(file_names[0])
df1 = pd.read_csv(file_names[1])
df2 = pd.read_csv(file_names[2])
df3 = pd.read_csv(file_names[3])
frames = [df0, df1, df2, df3]
df = pd.concat(frames)

In [42]:
df.head()

,Agency,Associated Prime Vendor,Budget Code,Capital Project,Check Amount,Contract ID,Contract Purpose,Department,Document ID,Emerging Business,Expense Category,Fiscal year,Industry,Issue Date,M/WBE Category,Payee Name,Spending Category,Sub Contract Reference ID,Sub Vendor,Woman Owned Business
0,Transit Authority,NaN,"BUSB (MTA, NYCT: PURCHASE 126 HYBRID BUSES)",998CAP2024 005,99999999.95,CT199820231408687,Bus Purch Track Replace Switch Replace Rail Su...,400-998-169,20230231715-1-DSB-EFT,No,CAPITAL PURCHASED EQUIPMENT,2023,Standardized Services,2022-12-21,Individuals and Others,METROPOLITAN TRANSPORTATION AUTHORITY,Capital Contracts,NaN,No,No
1,Transit Authority,NaN,"SAPS (MTA / NYCT: SUBWAY ACTION PLAN, SIGNAL A)",998CAPSAP 003,99999999.59,CT199820191409664,Subway Action Plan Signal Improvement & Modern...,400-998-169,20190025861-1-015-MD1,No,IOTB CONSTRUCTION,2019,Standardized Services,2018-10-30,Individuals and Others,METROPOLITAN TRANSPORTATION AUTHORITY,Capital Contracts,NaN,No,No
2,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA22 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220448629-1-DSB-EFT,No,ADMINISTRATIVE EXPENSES,2022,NaN,2022-05-27,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No
3,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA21 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220448626-1-DSB-EFT,No,CONSTRUCTION-BUILDINGS,2022,NaN,2022-05-27,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No
4,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA21 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220354644-1-DSB-EFT,No,CONSTRUCTION-BUILDINGS,2022,NaN,2022-03-25,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No


In [16]:
# limited data cleaning 

df = df[df['Check Amount']!=99999999]

In [19]:
cols_dict = {}
for col in df.columns:
    if col == 'Check Amount':
        cols_dict[col] = 'sum'
    else:
        cols_dict[col] = 'first' # keep first value for all other cols

In [20]:
# groupby
projects_df = df.groupby(['Capital Project', 'Budget Code', 'Agency', 'Fiscal year'], as_index=False)

In [21]:
#projects_df.drop('Issue Date', axis=1, inplace=True)
print(projects_df.shape)
projects_df.head(5)

AttributeError: 'DataFrameGroupBy' object has no attribute 'shape'

In [28]:
agg_result = projects_df.agg({col: 'nunique' for col in df.columns})

mean_result = agg_result.mean()
distinct = []
nondistinct = []
for col in df.columns:
    if col not in ['Agency', 'Budget Code', 'Fiscal year', 'Capital Project', 'Issue Date', 'Check Amount']:
        if (mean_result[col] == 1 or mean_result[col] == 0):
            nondistinct.append((col))
        else:
            distinct.append((col,mean_result[col]))

print("Cols with nondistinct values across groups:")
print(nondistinct)
print("Cols with distinct values across groups and their mean:")
print(distinct)


Cols with nondistinct values across groups:
['Associated Prime Vendor', 'Emerging Business', 'Spending Category', 'Sub Contract Reference ID', 'Sub Vendor']
Cols with distinct values across groups and their mean:
[('Contract ID', 6.641508445572332), ('Contract Purpose', 0.5824185418437074), ('Department', 1.0234275869294436), ('Document ID', 8.128220201855582), ('Expense Category', 1.0182589665087367), ('Industry', 0.499371384002887), ('M/WBE Category', 1.2814802742628313), ('Payee Name', 2.3187374131287615), ('Woman Owned Business', 1.1440287300792755)]


In [ ]:
from pathlib import Path
agg_df = projects_df.agg(cols_dict)
agg_df.drop('Issue Date', axis=1, inplace=True)

agg_df.to_csv('checkbook_nyc_groupby_capital_project_fy.csv')
